In [64]:
import geopandas as gpd
import requests
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon
import fontawesome as fa 



In [210]:
search_query = "Pharmacie"
city_name = "Youssoufia"
region = "Marrakech-Safi"

url = f"https://nominatim.openstreetmap.org/search?q={search_query},{city_name},{region}&format=json"

# Fetch data from OpenStreetMap API (Nominatim)
response = requests.get(url)
data = response.json()

# Extract coordinates and hospital names
hospitals = []
for place in data:
    hospital_name = place.get("display_name", "Unknown Hospital")
    lon, lat = place["lon"], place["lat"]
    hospitals.append({"name": hospital_name, "lon": lon, "lat": lat})

# Create a GeoDataFrame with the geometry column
gdf = gpd.GeoDataFrame(hospitals, geometry=gpd.points_from_xy([float(hospital["lon"]) for hospital in hospitals], 
                                                             [float(hospital["lat"]) for hospital in hospitals]),
                       crs="EPSG:4326")

display(gdf)


,name,lon,lat,geometry
0,"Pharmacie Moderne الصيدلية الحديثة, RN11, Yous...",-8.5371963,32.2415859,POINT (-8.53720 32.24159)
1,"Pharmacie Dokkali صيدلية دكالي, RN11, Youssouf...",-8.5291738,32.2566127,POINT (-8.52917 32.25661)
2,"Pharmacie Ibn Sina صيدلية ابن سينا, RN11, Yous...",-8.5422871,32.239898,POINT (-8.54229 32.23990)
3,"Pharmacie Kachkat صيدلية كاشكاط, RN11, Youssou...",-8.5332465,32.2458067,POINT (-8.53325 32.24581)
4,"Pharmacie Al Qods صيدلية القدس, RN11, Youssouf...",-8.5358955,32.2538836,POINT (-8.53590 32.25388)
5,"Pharmacie Annour صيدلية النور, RN11, Youssoufi...",-8.5326451,32.237092,POINT (-8.53265 32.23709)
6,"Pharmacie Lahdir صيدلية لهضير, RN11, Youssoufi...",-8.532388,32.2489358,POINT (-8.53239 32.24894)
7,"Pharmacie Louis Gentil صيدلية لويس جونتي, RN11...",-8.5283145,32.246148,POINT (-8.52831 32.24615)
8,"Pharmacie Najmi صيدلية نجمي, RN11, Youssoufia,...",-8.5342218,32.2515064,POINT (-8.53422 32.25151)
9,"Pharmacie Al Amal صيدلية الأمل, RN11, Youssouf...",-8.5328651,32.2353451,POINT (-8.53287 32.23535)


In [161]:
gdf.to_csv('data.csv', index = False)

In [53]:
data = pd.read_csv("merged_data.csv")

# Create a GeoDataFrame with the geometry column
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data["lon"], data["lat"]), crs="EPSG:4326")

# Create a Folium map centered at the specified location
map_center = [32.25, -8.52]
my_map = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB positron')

# Create a MarkerCluster layer to group the markers
marker_cluster = MarkerCluster().add_to(my_map)

# Add markers to the map
for idx, row in gdf.iterrows():
    name = row["name"]
    lat = row["lat"]
    lon = row["lon"]
    popup_text = f"<b>{name}</b><br>Latitude: {lat}<br>Longitude: {lon}"
    
    # Check the name for different categories (case-insensitive)
    if "pharmacie" in row["name"].lower():
        icon_color = "green"
        icon_html = '<i class="fa fa-hospital" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "gare" in row["name"].lower():
        icon_color = "red"
        icon_html = '<i class="fa fa-train" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "youcode" in row["name"].lower():
        icon_color = "blue"
        icon_html = '<i class="fa fa-graduation-cap" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "masjid" in row["name"].lower():
        icon_color = "orange"
        icon_html = '<i class="fa fa-mosque" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "souk" in row["name"].lower():
        icon_color = "purple"
        icon_html = '<i class="fa fa-shopping-basket" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "hôtel" in row["name"].lower():
        icon_color = "brown"
        icon_html = '<i class="fa fa-hotel" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "jardin" in row["name"].lower():
        icon_color = "#66ff66"  
        icon_html = '<i class="fa fa-tree" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    else:
        icon_color = "gray"
        icon_html = '<i class="fa fa-info-circle" style="color: {}; font-size: 30px;"></i>'.format(icon_color)

    folium.Marker(location=[lat, lon], popup=popup_text,
                  icon=DivIcon(icon_size=(30, 30), icon_anchor=(15, 15), html=icon_html)).add_to(marker_cluster)

# Create a legend HTML
legend_html = """
    <div style="
        position: fixed;
        bottom: 50px;
        left: 50px;
        z-index: 1000;
        padding: 10px;
        border: 2px solid grey;
        background-color: white;
        font-size: 16px;
    ">
    <p><i class="fa fa-hospital" style="color:green; font-size: 30px;"></i>&nbsp;Pharmacie</p>
    <p><i class="fa fa-train" style="color:red; font-size: 30px;"></i>&nbsp;Gare</p>
    <p><i class="fa fa-graduation-cap" style="color:blue; font-size: 30px;"></i>&nbsp;YouCode</p>
    <p><i class="fa fa-mosque" style="color:orange; font-size: 30px;"></i>&nbsp;Masjid</p>
    <p><i class="fa fa-shopping-basket" style="color:purple; font-size: 30px;"></i>&nbsp;Souk</p>
    <p><i class="fa fa-hotel" style="color:brown; font-size: 30px;"></i>&nbsp;Hôtel</p>
    <p><i class="fa fa-tree" style="color:#66ff66; font-size: 30px;"></i>&nbsp;Jardin</p>
    </div>
"""

# Add legend to the map
my_map.get_root().html.add_child(folium.Element(legend_html))

# Display the map
my_map.save('map.html')


In [31]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon
from geopy.distance import geodesic

# Read data from CSV file and create GeoDataFrame
data = pd.read_csv("merged_data.csv")
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data['lon'], data['lat']), crs="EPSG:4326")

# Create a Folium map centered at the specified location
map_center = [32.25, -8.52]
my_map = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB positron')

# Create a MarkerCluster layer to group the markers
marker_cluster = MarkerCluster().add_to(my_map)

# Add markers to the map
for idx, row in gdf.iterrows():
    name = row["name"]
    lat = row["lat"]
    lon = row["lon"]
    popup_text = f"<b>{name}</b><br>Latitude: {lat}<br>Longitude: {lon}"
    
    # Check the name for different categories (case-insensitive)
    if "pharmacie" in row["name"].lower():
        icon_color = "green"
        icon_html = '<i class="fa fa-hospital" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "gare" in row["name"].lower():
        icon_color = "red"
        icon_html = '<i class="fa fa-train" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "youcode" in row["name"].lower():
        icon_color = "blue"
        icon_html = '<i class="fa fa-graduation-cap" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "masjid" in row["name"].lower():
        icon_color = "orange"
        icon_html = '<i class="fa fa-mosque" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "souk" in row["name"].lower():
        icon_color = "purple"
        icon_html = '<i class="fa fa-shopping-basket" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "hôtel" in row["name"].lower():
        icon_color = "brown"
        icon_html = '<i class="fa fa-hotel" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "jardin" in row["name"].lower():
        icon_color = "#66ff66"  
        icon_html = '<i class="fa fa-tree" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    else:
        icon_color = "gray"
        icon_html = '<i class="fa fa-info-circle" style="color: {}; font-size: 30px;"></i>'.format(icon_color)

    folium.Marker(location=[lat, lon], popup=popup_text,
                  icon=DivIcon(icon_size=(30, 30), icon_anchor=(15, 15), html=icon_html)).add_to(marker_cluster)

# Create a legend HTML
legend_html = """
    <div style="
        position: fixed;
        bottom: 50px;
        left: 50px;
        z-index: 1000;
        padding: 10px;
        border: 2px solid grey;
        background-color: white;
        font-size: 16px;
    ">
    <p><i class="fa fa-hospital" style="color:green; font-size: 30px;"></i>&nbsp;Pharmacie</p>
    <p><i class="fa fa-train" style="color:red; font-size: 30px;"></i>&nbsp;Gare</p>
    <p><i class="fa fa-graduation-cap" style="color:blue; font-size: 30px;"></i>&nbsp;YouCode</p>
    <p><i class="fa fa-mosque" style="color:orange; font-size: 30px;"></i>&nbsp;Masjid</p>
    <p><i class="fa fa-shopping-basket" style="color:purple; font-size: 30px;"></i>&nbsp;Souk</p>
    <p><i class="fa fa-hotel" style="color:brown; font-size: 30px;"></i>&nbsp;Hôtel</p>
    <p><i class="fa fa-tree" style="color:#66ff66; font-size: 30px;"></i>&nbsp;Jardin</p>
    </div>
"""

# Add legend to the map
my_map.get_root().html.add_child(folium.Element(legend_html))

# Function to calculate the distance between two points
def calculate_distance(lat1, lon1, lat2, lon2):
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    distance = geodesic(coords_1, coords_2).meters
    return distance

# Sample coordinates for the line
line_coords_list = [
    (32.24708348779245, -8.521690026677078),
    (32.24759156175007, -8.52239940630048),
    (32.24788883774751, -8.522610302946163),
    (32.24803573612723, -8.522673451330093),
    (32.24865677734235, -8.522647636101809),
    (32.2473097456003, -8.532827815847874),
    (32.247620629637694, -8.533490720079246),
    (32.2470999145056, -8.533883469478099)
]

# Draw the line and add a popup with the distance
line_coords_list = [(lat, lon) for lon, lat in line_coords_list]  # Swap coordinates to (lat, lon) format
line = folium.PolyLine(locations=line_coords_list, color='blue', weight=3, opacity=0.7)
line.add_child(folium.Popup(f"Distance: {calculate_distance(*line_coords_list[0], *line_coords_list[-1]):.2f} meters")).add_to(my_map)

# Add legend to the map
my_map.get_root().html.add_child(folium.Element(legend_html))

# Display the map
my_map.save('map.html')


In [33]:
import csv

# GeoJSON data (as a dictionary)
geojson_data = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "coordinates": [
                    [-8.521690026677078, 32.24708348779245],
                    [-8.52239940630048, 32.24759156175007],
                    [-8.522610302946163, 32.24788883774751],
                    [-8.522673451330093, 32.24803573612723],
                    [-8.522647636101809, 32.24865677734235],
                    [-8.532827815847874, 32.2473097456003],
                    [-8.533490720079246, 32.247620629637694],
                    [-8.533883469478099, 32.2470999145056]
                ],
                "type": "LineString"
            },
            "id": 0
        }
    ]
}

# Extract coordinates and ID from the GeoJSON data
coordinates = geojson_data["features"][0]["geometry"]["coordinates"]
line_id = geojson_data["features"][0]["id"]

# Prepare the CSV data
csv_data = [["Latitude", "Longitude", "ID"]]
for lat, lon in coordinates:
    csv_data.append([lat, lon, line_id])

# Write the CSV data to a file
csv_file_path = "converted_data.csv"
with open(csv_file_path, "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows(csv_data)

print(f"CSV data has been saved to {csv_file_path}.")


CSV data has been saved to converted_data.csv.


In [51]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon
from geopy.distance import geodesic

# Function to calculate a sample route (for demonstration purposes)
def calculate_sample_route(start_coords, end_coords):
    # This is a sample route for demonstration purposes
    # Replace this with the actual routing algorithm or API for real routing
    route_coords = [start_coords, end_coords]
    return route_coords

# Function to display the route on the map
def display_route(route_coords, my_map, label):
    folium.PolyLine(route_coords, color='blue', weight=3, opacity=0.7, popup=label).add_to(my_map)

# Read data from CSV file and create GeoDataFrame
data = pd.read_csv("merged_data.csv")
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data['lon'], data['lat']), crs="EPSG:4326")

# Create a Folium map centered at the specified location
map_center = [32.25, -8.52]
my_map = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB positron')

# Create a MarkerCluster layer to group the markers
marker_cluster = MarkerCluster().add_to(my_map)

# Add markers to the map
for idx, row in gdf.iterrows():
    name = row["name"]
    lat = row["lat"]
    lon = row["lon"]
    popup_text = f"<b>{name}</b><br>Latitude: {lat}<br>Longitude: {lon}"

    # Check the name for different categories (case-insensitive)
    if "pharmacie" in row["name"].lower():
        icon_color = "green"
        icon_html = '<i class="fa fa-hospital" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "gare" in row["name"].lower():
        icon_color = "red"
        icon_html = '<i class="fa fa-train" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "youcode" in row["name"].lower():
        icon_color = "blue"
        icon_html = '<i class="fa fa-graduation-cap" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "masjid" in row["name"].lower():
        icon_color = "orange"
        icon_html = '<i class="fa fa-mosque" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "souk" in row["name"].lower():
        icon_color = "purple"
        icon_html = '<i class="fa fa-shopping-basket" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "hôtel" in row["name"].lower():
        icon_color = "brown"
        icon_html = '<i class="fa fa-hotel" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "jardin" in row["name"].lower():
        icon_color = "#66ff66"
        icon_html = '<i class="fa fa-tree" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    else:
        icon_color = "gray"
        icon_html = '<i class="fa fa-info-circle" style="color: {}; font-size: 30px;"></i>'.format(icon_color)

    folium.Marker(location=[lat, lon], popup=folium.Popup(popup_text),
                  icon=DivIcon(icon_size=(30, 30), icon_anchor=(15, 15), html=icon_html)).add_to(marker_cluster)

# Function to calculate the distance between two points
def calculate_distance(lat1, lon1, lat2, lon2):
    coords_1 = (lon1,lat1 )
    coords_2 = (lon2,lat2 )
    distance = geodesic(coords_1, coords_2).meters
    return distance

# Sample coordinates for the routes
route_coords_1 = [
    (-8.5224114,32.2471087),  # YouCode
    (-8.5314156,32.2450427)          # Jardin
]

route_coords_2 = [
    (-8.5224114,32.24710879),  # YouCode
    (-8.53456296286046,32.247460450000005)                  # Gare
]

route_coords_3 = [
    (-8.5224114,32.2471087),  # YouCode
    (-8.539445,32.2445097)          # Hôtel de Youssoufia
]

# Display the routes on the map with distances as popups
display_route(route_coords_1, my_map, f"Distance to Jardin: {calculate_distance(*route_coords_1[0], *route_coords_1[-1]):.2f} meters")
display_route(route_coords_2, my_map, f"Distance to Gare: {calculate_distance(*route_coords_2[0], *route_coords_2[-1]):.2f} meters")
display_route(route_coords_3, my_map, f"Distance to Hôtel de Youssoufia: {calculate_distance(*route_coords_3[0], *route_coords_3[-1]):.2f} meters")

# Add the embedded Google Maps iframes to markers and then to the map
embedded_map_html_1 = '<iframe src="https://www.google.com/maps/embed?pb=!1m28!1m12!1m3!1d5228.672031687831!2d-8.531014861262369!3d32.245415769729235!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!4m13!3e0!4m5!1s0xdaefdd4fcbbdbc1%3A0x846cbd9f328a7bdb!2sYOUCODE%2C%20%C3%89cole%20YOUCODE%2C%20Youssoufia!3m2!1d32.2474527!2d-8.5216498!4m5!1s0xdaefd5e33e0348b%3A0x25cd075b5f5eb949!2z2K3Yr9mK2YLYqSDYtNin2LHYuSDYp9mE2K3Ys9mGINin2YTYqtin2YbZiiBKYXJkaW4gYm91bGV2YXJkIGhhc3NhbiBJSSwgNkZWQytWN1YsIFlvdXNzb3VmaWE!3m2!1d32.244734799999996!2d-8.5293197!5e1!3m2!1sen!2sma!4v1690887263856!5m2!1sen!2sma" width="600" height="450" style="border:0;" allowfullscreen="" loading="lazy" referrerpolicy="no-referrer-when-downgrade"></iframe>'
embedded_map_html_2 = '<iframe src="https://www.google.com/maps/embed?pb=!1m28!1m12!1m3!1d5228.4647038498015!2d-8.532524111261663!3d32.249016969363005!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!4m13!3e0!4m5!1s0xdaefdd4fcbbdbc1%3A0x846cbd9f328a7bdb!2sYOUCODE%2C%20%C3%89cole%20YOUCODE%2C%20Youssoufia!3m2!1d32.2474527!2d-8.5216498!4m5!1s0xdaefcd343bb09b7%3A0x27366ca5953ccbc9!2sYoussoufia%2C%20R201%2C%20Youssoufia!3m2!1d32.2476777!2d-8.5344547!5e1!3m2!1sen!2sma!4v1690898593825!5m2!1sen!2sma" width="600" height="450" style="border:0;" allowfullscreen="" loading="lazy" referrerpolicy="no-referrer-when-downgrade"></iframe>'
embedded_map_html_3 = '<iframe src="https://www.google.com/maps/embed?pb=!1m28!1m12!1m3!1d5228.529416855218!2d-8.533792211261867!3d32.247892969477356!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!4m13!3e0!4m5!1s0xdaefdd4fcbbdbc1%3A0x846cbd9f328a7bdb!2sYOUCODE%2C%20%C3%89cole%20YOUCODE%2C%20Youssoufia!3m2!1d32.2474527!2d-8.5216498!4m5!1s0xdaefcd3bf112d51%3A0x86cd313734ec4563!2s6FV7%2BRV5%20H%C3%B4tel%20de%20Youssoufia%2C%20R201%2C%20Youssoufia!3m2!1d32.2445101!2d-8.535264699999999!5e1!3m2!1sen!2sma!4v1690898790418!5m2!1sen!2sma" width="600" height="450" style="border:0;" allowfullscreen="" loading="lazy" referrerpolicy="no-referrer-when-downgrade"></iframe>'
folium.Marker([32.244734799999996, -8.5293197], popup=folium.Popup(embedded_map_html_1, max_width=700), icon=None).add_to(my_map)
folium.Marker([32.2476777, -8.5344547], popup=folium.Popup(embedded_map_html_2, max_width=700), icon=None).add_to(my_map)
folium.Marker([32.2445101, -8.535264699999999], popup=folium.Popup(embedded_map_html_3, max_width=700), icon=None).add_to(my_map)

# Create a legend HTML
legend_html = """
    <div style="
        position: fixed;
        bottom: 50px;
        left: 50px;
        z-index: 1000;
        padding: 10px;
        border: 2px solid grey;
        background-color: white;
        font-size: 16px;
    ">
    <p><i class="fa fa-hospital" style="color:green; font-size: 30px;"></i>&nbsp;Pharmacie</p>
    <p><i class="fa fa-train" style="color:red; font-size: 30px;"></i>&nbsp;Gare</p>
    <p><i class="fa fa-graduation-cap" style="color:blue; font-size: 30px;"></i>&nbsp;YouCode</p>
    <p><i class="fa fa-mosque" style="color:orange; font-size: 30px;"></i>&nbsp;Masjid</p>
    <p><i class="fa fa-shopping-basket" style="color:purple; font-size: 30px;"></i>&nbsp;Souk</p>
    <p><i class="fa fa-hotel" style="color:brown; font-size: 30px;"></i>&nbsp;Hôtel</p>
    <p><i class="fa fa-tree" style="color:#66ff66; font-size: 30px;"></i>&nbsp;Jardin</p>
    </div>
"""

# Add legend to the map
my_map.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
my_map.save('map_with_routes_and_embedded_maps.html')


In [72]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon
import googlemaps
from geopy.distance import geodesic

def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

# Read data from CSV file and create GeoDataFrame
data = pd.read_csv("merged_data.csv")
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data['lon'], data['lat']), crs="EPSG:4326")

# Create a Folium map centered at the specified location
map_center = [32.25, -8.52]
my_map = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB positron')

# Create a MarkerCluster layer to group the markers
marker_cluster = MarkerCluster().add_to(my_map)

# Add markers to the map
for idx, row in gdf.iterrows():
    name = row["name"]
    lat = row["lat"]
    lon = row["lon"]
    popup_text = f"<b>{name}</b><br>Latitude: {lat}<br>Longitude: {lon}"

    # Check the name for different categories (case-insensitive)
    if "pharmacie" in row["name"].lower():
        icon_color = "green"
        icon_html = '<i class="fa fa-hospital" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "gare" in row["name"].lower():
        icon_color = "red"
        icon_html = '<i class="fa fa-train" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "youcode" in row["name"].lower():
        icon_color = "blue"
        icon_html = '<i class="fa fa-graduation-cap" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "masjid" in row["name"].lower():
        icon_color = "orange"
        icon_html = '<i class="fa fa-mosque" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "souk" in row["name"].lower():
        icon_color = "purple"
        icon_html = '<i class="fa fa-shopping-basket" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "hôtel" in row["name"].lower():
        icon_color = "brown"
        icon_html = '<i class="fa fa-hotel" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "jardin" in row["name"].lower():
        icon_color = "#66ff66"
        icon_html = '<i class="fa fa-tree" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    else:
        icon_color = "gray"
        icon_html = '<i class="fa fa-info-circle" style="color: {}; font-size: 30px;"></i>'.format(icon_color)

    folium.Marker(location=[lat, lon], popup=folium.Popup(popup_text),
                  icon=DivIcon(icon_size=(30, 30), icon_anchor=(15, 15), html=icon_html)).add_to(marker_cluster)

# List of place names to create the LineString
place_names = ["Youcode", "masjid niour", "souk baladi", "Hôtel de Youssoufia", "Jardin boulevard hassan II"]

# Filter the data to include only the specified place names
line_data = gdf[gdf["name"].isin(place_names)]

# Create a LineString to draw on the map
line_coords = [(lat, lon) for lat, lon in zip(line_data["lat"], line_data["lon"])]
line = folium.PolyLine(locations=line_coords, color='blue', weight=3, opacity=0.7)

# Print the distance between each pair of consecutive points on the line
for i in range(len(line_coords) - 1):
    distance_between_points = calculate_distance(line_coords[i], line_coords[i + 1])
    print(f"Distance between {place_names[i]} and {place_names[i+1]}: {distance_between_points:.2f} kilometers")

# Add a popup with the total distance to the line
line_popup_text = "Line distance"
line.add_child(folium.Popup(line_popup_text)).add_to(my_map)

# Create a legend HTML (same as before)
legend_html = """
    <div style="
        position: fixed;
        bottom: 50px;
        left: 50px;
        z-index: 1000;
        padding: 10px;
        border: 2px solid grey;
        background-color: white;
        font-size: 16px;
    ">
    <p><i class="fa fa-hospital" style="color:green; font-size: 30px;"></i>&nbsp;Pharmacie</p>
    <p><i class="fa fa-train" style="color:red; font-size: 30px;"></i>&nbsp;Gare</p>
    <p><i class="fa fa-graduation-cap" style="color:blue; font-size: 30px;"></i>&nbsp;YouCode</p>
    <p><i class="fa fa-mosque" style="color:orange; font-size: 30px;"></i>&nbsp;Masjid</p>
    <p><i class="fa fa-shopping-basket" style="color:purple; font-size: 30px;"></i>&nbsp;Souk</p>
    <p><i class="fa fa-hotel" style="color:brown; font-size: 30px;"></i>&nbsp;Hôtel</p>
    <p><i class="fa fa-tree" style="color:#66ff66; font-size: 30px;"></i>&nbsp;Jardin</p>
    </div>
"""

# Add the legend HTML to the map
my_map.get_root().html.add_child(folium.Element(legend_html))

# Use Google Maps API to show directions and distance by car
gmaps = googlemaps.Client(key='YOUR_GOOGLE_MAPS_API_KEY')
for i in range(len(place_names) - 1):
    start = (line_coords[i][0], line_coords[i][1])
    end = (line_coords[i + 1][0], line_coords[i + 1][1])
    directions = gmaps.directions(start, end, mode='driving')
    if directions:
        distance_by_car = directions[0]['legs'][0]['distance']['text']
        print(f"Distance by car between {place_names[i]} and {place_names[i+1]}: {distance_by_car}")

# Save the map to an HTML file
my_map.save('map_with_routes_and_embedded_maps.html')


Distance between Youcode and masjid niour: 2.30 kilometers
Distance between masjid niour and souk baladi: 2.57 kilometers
Distance between souk baladi and Hôtel de Youssoufia: 0.77 kilometers
Distance between Hôtel de Youssoufia and Jardin boulevard hassan II: 0.89 kilometers


In [64]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon
from geopy.distance import geodesic
from IPython.display import display
import ipywidgets as widgets

def calculate_line_distance(coords_list):
    total_distance = 0.0
    for i in range(len(coords_list) - 1):
        point1 = (coords_list[i][1], coords_list[i][0])  # Swap coordinates to (lat, lon) format
        point2 = (coords_list[i + 1][1], coords_list[i + 1][0])  # Swap coordinates to (lat, lon) format
        total_distance += geodesic(point1, point2).kilometers
    return total_distance

# Read data from CSV file and create GeoDataFrame
data = pd.read_csv("merged_data.csv")
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data['lon'], data['lat']), crs="EPSG:4326")

# Create a Folium map centered at the specified location
map_center = [32.25, -8.52]
my_map = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB positron')

# Create a MarkerCluster layer to group the markers
marker_cluster = MarkerCluster().add_to(my_map)

# Add markers to the map
for idx, row in gdf.iterrows():
    name = row["name"]
    lat = row["lat"]
    lon = row["lon"]
    popup_text = f"<b>{name}</b><br>Latitude: {lat}<br>Longitude: {lon}"

    # Check the name for different categories (case-insensitive)
    if "pharmacie" in row["name"].lower():
        icon_color = "green"
        icon_html = '<i class="fa fa-hospital" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "gare" in row["name"].lower():
        icon_color = "red"
        icon_html = '<i class="fa fa-train" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "youcode" in row["name"].lower():
        icon_color = "blue"
        icon_html = '<i class="fa fa-graduation-cap" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "masjid" in row["name"].lower():
        icon_color = "orange"
        icon_html = '<i class="fa fa-mosque" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "souk" in row["name"].lower():
        icon_color = "purple"
        icon_html = '<i class="fa fa-shopping-basket" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "hôtel" in row["name"].lower():
        icon_color = "brown"
        icon_html = '<i class="fa fa-hotel" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "jardin" in row["name"].lower():
        icon_color = "#66ff66"
        icon_html = '<i class="fa fa-tree" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    else:
        icon_color = "gray"
        icon_html = '<i class="fa fa-info-circle" style="color: {}; font-size: 30px;"></i>'.format(icon_color)

    folium.Marker(location=[lat, lon], popup=folium.Popup(popup_text),
                  icon=DivIcon(icon_size=(30, 30), icon_anchor=(15, 15), html=icon_html)).add_to(marker_cluster)

# Coordinates for the initial LineString
line_coords = [
    (-8.52187, 32.24723),
    # ... (the rest of the coordinates)
    (-8.53425, 32.24758)
]

# Swap coordinates to (lat, lon) format for LineString
line_coords = [(lat, lon) for lon, lat in line_coords]
line = folium.PolyLine(locations=line_coords, color='blue', weight=3, opacity=0.7)
line_length = calculate_line_distance(line_coords)  # Calculate distance of the initial LineString
line_popup_text = f"Distance: {line_length:.2f} kilometers"
line.add_child(folium.Popup(line_popup_text))

# Create a legend HTML
legend_html = """
    <div style="
        position: fixed;
        bottom: 50px;
        left: 50px;
        z-index: 1000;
        padding: 10px;
        border: 2px solid grey;
        background-color: white;
        font-size: 16px;
    ">
    <p><i class="fa fa-hospital" style="color:green; font-size: 30px;"></i>&nbsp;Pharmacie</p>
    <p><i class="fa fa-train" style="color:red; font-size: 30px;"></i>&nbsp;Gare</p>
    <p><i class="fa fa-graduation-cap" style="color:blue; font-size: 30px;"></i>&nbsp;YouCode</p>
    <p><i class="fa fa-mosque" style="color:orange; font-size: 30px;"></i>&nbsp;Masjid</p>
    <p><i class="fa fa-shopping-basket" style="color:purple; font-size: 30px;"></i>&nbsp;Souk</p>
    <p><i class="fa fa-hotel" style="color:brown; font-size: 30px;"></i>&nbsp;Hôtel</p>
    <p><i class="fa fa-tree" style="color:#66ff66; font-size: 30px;"></i>&nbsp;Jardin</p>
    </div>
"""

## Add the LineString to the FeatureGroup
line_group.add_child(line)

# Function to update the LineString based on user input
def update_line(change):
    new_coords = [tuple(map(float, coord.split(","))) for coord in coord_input.value.split("\n") if coord.strip()]
    global line_coords, line, line_length
    line_coords = [(lat, lon) for lon, lat in new_coords]

    # Remove the previous LineString from the map
    line_group.remove_layer(line)

    # Create a new LineString and calculate its distance
    line = folium.PolyLine(locations=line_coords, color='blue', weight=3, opacity=0.7)
    line_length = calculate_line_distance(line_coords)
    line_popup_text = f"Distance: {line_length:.2f} kilometers"
    line.add_child(folium.Popup(line_popup_text))

    # Add the new LineString to the FeatureGroup and add the FeatureGroup to the map
    line_group.add_child(line)
    my_map.add_child(line_group)


# Function to handle the form submission
def on_submit(btn):
    update_line(None)

# Attach the on_click function to the submit button
submit_button.on_click(on_submit)

# Display the input form, submit button, and the map
display(coord_input)
display(submit_button)
display(my_map)

Textarea(value='-8.52187, 32.24723\n-8.53425, 32.24758', description='Coordinates:', layout=Layout(width='50%'…

Button(description='Submit', style=ButtonStyle())

AttributeError: 'FeatureGroup' object has no attribute 'remove_layer'

AttributeError: 'FeatureGroup' object has no attribute 'remove_layer'

AttributeError: 'FeatureGroup' object has no attribute 'remove_layer'

AttributeError: 'FeatureGroup' object has no attribute 'remove_layer'

AttributeError: 'FeatureGroup' object has no attribute 'remove_layer'

AttributeError: 'FeatureGroup' object has no attribute 'remove_layer'